## Component One: Predicting Customer Spending Behaviour in Streaming Services.

In [2]:
# Importing Libraries
import pandas as pd
import numpy as np

# Reading the streaming services dataset
# This dataset contains customer information that we'll use to predict spending behaviour
df = pd.read_csv('/Users/dannywu/Library/CloudStorage/OneDrive-hull.ac.uk/AI/FinalProject/Streaming.csv')

print("Dataset shape:", df.shape)
print("\nFirst few rows:")
df.head()

Dataset shape: (5000, 12)

First few rows:


,Customer_ID,Age,Gender,Subscription_Length,Region,Payment_Method,Support_Tickets_Raised,Satisfaction_Score,Discount_Offered,Last_Activity,Monthly_Spend,Churned
0,CUST000001,56.0,Male,54,South,PayPal,0,9.0,6.42,319,62.11,1
1,CUST000002,69.0,Female,21,East,Debit Card,1,2.0,13.77,166,37.27,1
2,CUST000003,46.0,Female,49,East,PayPal,3,8.0,19.91,207,61.82,0
3,CUST000004,32.0,Male,47,West,Debit Card,3,1.0,13.39,108,40.96,1
4,CUST000005,60.0,Male,6,East,Credit Card,2,NaN,13.18,65,45.97,0
